In [15]:
import os

def change_class_id(yolo_file_path):
    with open(yolo_file_path, 'r') as file:
        lines = file.readlines()

    with open(yolo_file_path, 'w') as file:
        for line in lines:
            parts = line.split()
            if len(parts) >= 5:
                # Change the class ID (assuming it is the first value in each line)
                parts[0] = '1'
                new_line = ' '.join(parts) + '\n'
                file.write(new_line)
            else:
                # If the line does not have enough values, write it as is
                file.write(line)

# Specify the path to the folder containing YOLO files
yolo_folder = "D:\kuliah\Magang_Winteq\Project\Data Baru dengan Annotations\Cap\Cap_Annotations_YOLOv5"

# Iterate through YOLO files in the folder and change class ID
for yolo_file in os.listdir(yolo_folder):
    if yolo_file.endswith(".txt"):
        yolo_path = os.path.join(yolo_folder, yolo_file)
        change_class_id(yolo_path)

print("Class IDs changed successfully.")


Class IDs changed successfully.


In [14]:
import os
import xml.etree.ElementTree as ET

def convert_xml_to_txt(xml_file, txt_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(txt_file, 'w') as txt_file:
        for elem in root.iter():
            # Extract relevant information and write to the text file
            if elem.text:
                txt_file.write(elem.text + '\n')

# Specify the path to the folder containing XML files
xml_folder = r"D:\kuliah\Magang_Winteq\Project\Nitip"

# Specify the path to the folder where TXT files will be saved
txt_folder = r"D:\kuliah\Magang_Winteq\Project\Nitip2"

# Ensure the output directory exists
os.makedirs(txt_folder, exist_ok=True)

# Iterate through XML files in the folder and convert each to TXT
for xml_file in os.listdir(xml_folder):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(xml_folder, xml_file)
        txt_file = os.path.join(txt_folder, os.path.splitext(xml_file)[0] + ".txt")
        convert_xml_to_txt(xml_path, txt_file)

In [13]:
import os
import xml.etree.ElementTree as ET

def convert_xml_to_yolo(xml_file, yolo_file, class_mapping):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(yolo_file, 'w') as yolo_file:
        for obj in root.iter('object'):
            class_name = obj.find('name').text

            # Check if class_name exists in the class_mapping dictionary
            if class_name not in class_mapping:
                print(f"Warning: Unknown class '{class_name}'. Skipping...")
                continue

            class_id = class_mapping[class_name]

            bndbox = obj.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            width = xmax - xmin
            height = ymax - ymin
            center_x = xmin + width / 2
            center_y = ymin + height / 2

            # Normalize coordinates
            image_width = float(root.find('size').find('width').text)
            image_height = float(root.find('size').find('height').text)

            center_x /= image_width
            center_y /= image_height
            width /= image_width
            height /= image_height

            yolo_file.write(f"{class_id} {center_x} {center_y} {width} {height}\n")

# Specify the path to the folder containing XML files
xml_folder = r"D:\kuliah\Magang_Winteq\Project\Nitip"

# Specify the path to the folder where YOLO files will be saved
yolo_folder = r"D:\kuliah\Magang_Winteq\Project\Nitip2"

# Ensure the output directory exists
os.makedirs(yolo_folder, exist_ok=True)

# Class mapping (replace with your own class names and IDs)
class_mapping = {"helmet": 0}

# Iterate through XML files in the folder and convert each to YOLO format
for xml_file in os.listdir(xml_folder):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(xml_folder, xml_file)
        yolo_file = os.path.join(yolo_folder, os.path.splitext(xml_file)[0] + ".txt")
        convert_xml_to_yolo(xml_path, yolo_file, class_mapping)

In [1]:
import imghdr
import os
import cv2
import shutil
import math
import random as rd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
batch_size = 32
img_height = 180
img_width = 180
data_dir = "D:\\kuliah\\Magang_Winteq\\Project\\dataV2"

# Define valid image extensions
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [3]:
# Function to check image validity
def is_valid_image(img_path):
    try:
        # Try to read the image using OpenCV
        img = cv2.imread(img_path)
        if img is None:
            return False
        # Check image file format using imghdr
        tip = imghdr.what(img_path)
        return tip in image_exts
    except Exception as e:
        return False

# Clean the data directory
for image_class in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, image_class)
    for image in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image)
        if not is_valid_image(image_path):
            print(f'Removing invalid image: {image_path}')
            os.remove(image_path)

In [4]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Resizing(img_height, img_width),  # Resize with a slight increase in size
    tf.keras.layers.Rescaling(1./255),  # Rescale pixel values to [0,1]
])

# Load and preprocess training data with data augmentation
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Load validation data
val_test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Split validation data into validation and test sets
val_ds = val_test_ds.take(len(val_test_ds) // 2)
test_ds = val_test_ds.skip(len(val_test_ds) // 2)

Found 2716 files belonging to 2 classes.
Using 2173 files for training.
Found 2716 files belonging to 2 classes.
Using 543 files for validation.


In [5]:
# Model architecture
num_classes = len(train_ds.class_names)
model = Sequential([
    layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Model training
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
)

Epoch 1/20
68/68 [==============================] - 59s 821ms/step - loss: 55.0914 - accuracy: 0.6673 - val_loss: 0.4997 - val_accuracy: 0.7344
Epoch 2/20
68/68 [==============================] - 57s 833ms/step - loss: 0.4544 - accuracy: 0.7671 - val_loss: 0.4411 - val_accuracy: 0.7891
Epoch 3/20
68/68 [==============================] - 55s 803ms/step - loss: 0.4134 - accuracy: 0.8030 - val_loss: 0.4623 - val_accuracy: 0.8203
Epoch 4/20
68/68 [==============================] - 57s 818ms/step - loss: 0.3459 - accuracy: 0.8389 - val_loss: 0.5261 - val_accuracy: 0.8086
Epoch 5/20
68/68 [==============================] - 55s 804ms/step - loss: 0.2883 - accuracy: 0.8656 - val_loss: 0.4846 - val_accuracy: 0.7930
Epoch 6/20
68/68 [==============================] - 56s 814ms/step - loss: 0.3107 - accuracy: 0.8601 - val_loss: 0.5579 - val_accuracy: 0.8281
Epoch 7/20
68/68 [==============================] - 55s 799ms/step - loss: 0.3211 - accuracy: 0.8670 - val_loss: 0.8006 - val_accuracy: 0.816

In [7]:
# Model evaluation on the test set
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

9/9 [==============================] - 6s 313ms/step - loss: 1.1363 - accuracy: 0.8676
Test Accuracy: 86.76%


In [21]:
class_names = train_ds.class_names

In [23]:
import cv2
import numpy as np

# Fungsi untuk mendeteksi dan menampilkan kotak pada objek menggunakan model
def detect_and_display(frame, model, class_names):
    # Praproses frame
    resized_frame = cv2.resize(frame, (img_width, img_height))
    input_data = np.expand_dims(resized_frame, axis=0) / 255.0

    # Prediksi menggunakan model
    predictions = model.predict(input_data)
    score = tf.nn.softmax(predictions[0])
    class_index = np.argmax(score)
    confidence = 100 * np.max(score)

    # Jika kepercayaan (confidence) lebih dari batas tertentu, tampilkan kotak dan label
    if confidence > 75:
        label = class_names[class_index]

        # Koordinat kotak
        box_color = (0, 255, 0)  # Warna hijau
        box_thickness = 2

        # Ambil ukuran frame
        height, width, _ = frame.shape

        # Ambil koordinat kotak hasil prediksi
        xmin = int(predictions[0][0] * width)
        ymin = int(predictions[0][1] * height)
        xmax = int(predictions[0][2] * width)
        ymax = int(predictions[0][3] * height)

        # Gambar kotak pada frame
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), box_color, box_thickness)

        # Tambahkan label pada kotak
        label_position = (xmin, ymin - 10)  # Sedikit di atas kotak
        cv2.putText(frame, f'{label} ({confidence:.2f}%)', label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    return frame

# Fungsi untuk membaca video dari kamera
def main():
    cap = cv2.VideoCapture(0)  # 0 untuk kamera bawaan

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Deteksi dan tampilkan hasil pada frame
        frame_with_detection = detect_and_display(frame, model, class_names)

        # Tampilkan frame
        cv2.imshow('Safety Helmet and Cap Detection', frame_with_detection)

        # Hentikan aplikasi jika tombol 'q' ditekan
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Tutup video stream dan jendela OpenCV
    cap.release()
    cv2.destroyAllWindows()

# Panggil fungsi main
main()

1/1 [==============================] - 0s 54ms/step


In [17]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Path ke folder test cap
test_cap_path = "D:\kuliah\Magang_Winteq\Project\cap_valid"

# Inisialisasi variabel
correct_predictions = 0
total_predictions = 0

# Iterasi melalui gambar-gambar di folder test
for filename in os.listdir(test_cap_path):
    if filename.endswith(".jpg"):
        # Path lengkap ke gambar
        img_path = os.path.join(test_cap_path, filename)

        # Load dan preprocess gambar
        img = image.load_img(img_path, target_size=(img_height, img_width))
        img_array = image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Tambahkan dimensi batch
        img_array /= 255.0  # Normalisasi

        # Prediksi menggunakan model
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions[0])

        # Cek apakah prediksi benar
        true_class = 0  # Ganti dengan indeks kelas "cap"
        if predicted_class == true_class:
            correct_predictions += 1

        total_predictions += 1

# Hitung akurasi
accuracy = correct_predictions / total_predictions
print(f"Accuracy on test dataset: {accuracy * 100:.2f}%")

Accuracy on test dataset: 0.00%


# **YOLO**